In [2]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.autograd import Variable
from torch import optim
import torch.nn as nn
import torch.nn.functional as F

learning_rate = 1e-3
batch_size = 100
epoches = 50

trans_img = transforms.Compose([
        transforms.ToTensor()
    ])

trainset = MNIST('./data', train=True, transform=trans_img)
testset = MNIST('./data', train=False, transform=trans_img)

trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)

# build network
class Lenet(nn.Module):
    def __init__(self):
        super(Lenet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 6, 3, stride=1, padding=1),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 5, stride=1, padding=0),
            nn.MaxPool2d(2, 2)
        )
        
        self.fc = nn.Sequential(
            nn.Linear(400, 120),
            nn.Linear(120, 84),
            nn.Linear(84, 10)
        )
        
        
    def forward(self, x):
        out = self.conv(x)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

lenet = Lenet()
lenet.cuda()

criterian = nn.CrossEntropyLoss(size_average=False)
optimizer = optim.SGD(lenet.parameters(), lr=learning_rate)

# train
for i in range(epoches):
    running_loss = 0.
    running_acc = 0.
    for (img, label) in trainloader:
        img = Variable(img).cuda()
        label = Variable(label).cuda()
        
        optimizer.zero_grad()
        output = lenet(img)
        loss = criterian(output, label)
        # backward
        loss.backward()
        optimizer.step()
        
        running_loss += loss.data[0]
        _, predict = torch.max(output, 1)
        correct_num = (predict == label).sum()
        running_acc += correct_num.data[0]
    
    running_loss /= len(trainset)
    running_acc /= len(trainset)
    print("[%d/%d] Loss: %.5f, Acc: %.2f" %(i+1, epoches, running_loss, 100*running_acc))

[1/50] Loss: 0.41802, Acc: 87.36
[2/50] Loss: 0.10981, Acc: 96.65
[3/50] Loss: 0.08377, Acc: 97.43
[4/50] Loss: 0.06959, Acc: 97.82
[5/50] Loss: 0.06073, Acc: 98.08
[6/50] Loss: 0.05478, Acc: 98.28
[7/50] Loss: 0.04953, Acc: 98.46
[8/50] Loss: 0.04746, Acc: 98.49
[9/50] Loss: 0.04319, Acc: 98.64
[10/50] Loss: 0.04073, Acc: 98.72
[11/50] Loss: 0.03686, Acc: 98.80
[12/50] Loss: 0.03555, Acc: 98.88
[13/50] Loss: 0.03413, Acc: 98.89
[14/50] Loss: 0.03213, Acc: 98.95
[15/50] Loss: 0.03059, Acc: 99.00
[16/50] Loss: 0.02950, Acc: 99.05
[17/50] Loss: 0.02880, Acc: 99.05
[18/50] Loss: 0.02750, Acc: 99.13
[19/50] Loss: 0.02475, Acc: 99.19
[20/50] Loss: 0.02391, Acc: 99.21
[21/50] Loss: 0.02380, Acc: 99.19
[22/50] Loss: 0.02223, Acc: 99.27
[23/50] Loss: 0.02041, Acc: 99.32
[24/50] Loss: 0.02106, Acc: 99.30
[25/50] Loss: 0.01992, Acc: 99.34
[26/50] Loss: 0.01892, Acc: 99.36
[27/50] Loss: 0.01928, Acc: 99.37
[28/50] Loss: 0.01753, Acc: 99.43
[29/50] Loss: 0.01797, Acc: 99.42
[30/50] Loss: 0.01573, 

In [6]:
# evaluate
lenet.eval()

testloss = 0.
testacc = 0.
for (img, label) in testloader:
    img = Variable(img).cuda()
    label = Variable(label).cuda()
    
    output = lenet(img)
    loss = criterian(output, label)
    testloss += loss.data[0]
    _, predict = torch.max(output, 1)
    num_correct = (predict == label).sum()
    testacc += num_correct.data[0]

testloss /= len(testset)
testacc /= len(testset)
print("Test: Loss: %.5f, Acc: %.2f %%" %(testloss, 100*testacc))

Test: Loss: 0.08638, Acc: 98.43 %
